In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

avocado = pd.read_csv('avocado.csv')
pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 280)
avocado['Month'] = (avocado['Date'].str[5:]).str[:2]
print(avocado.head())
print(avocado['year'].value_counts())   # Смотрим как рапределены данные по годам и видим, что за 2018 мало записей
print(avocado[avocado['year']==2018]['Month'].unique()) # данные за 2018 год не полные (есть только 3 месяца), уберем этот год из анализа
data = avocado[avocado['year'] < 2018]

# Работаем над столбцом AveragePrice (средняя цена за штуку)
# Посмотрим как меняетя средняя цена в зависимости от месяца
data.groupby('Month').mean()[['AveragePrice']].plot(xlabel='Month')
plt.show()
# Посмотрим как меняетя средняя цена в зависимости от месяца по годам, для этого сгруппируем и усредним ее по месяцам и годам
# Видно, что в течение 2017 года цена выросла, по сравнению с предыдущими годами
data_month = data.groupby(['year','Month']).mean()[['AveragePrice']].reset_index()
for x in data['year'].unique():
    plt.plot(data_month['Month'].unique(), data_month[data_month['year'] == x]['AveragePrice'])
    plt.title('Динамика изменения цен на авокадо по месяцам и годам')
    plt.xlabel('Месяц')
    plt.ylabel('Цена')
    plt.legend(['2015','2016','2017'])
plt.show()
# На боксплоте также видно, что среднее значение цены выросло в 2017, но при этом и увеличился размах между максимальным и минимальным значениями
data_box_ap = [data[data['year'] == 2015]['AveragePrice'], data[data['year'] == 2016]['AveragePrice'], data[data['year'] == 2017]['AveragePrice']]
plt.boxplot(data_box_ap, labels=['2015','2016','2017'])
plt.title('Среднее значение цены по годам')
plt.show()
# Посмотрим на показатель Total Volume(общие продажи)
# Построим боксплот в разрезе годов, на нем видно, что есть какие-то выбросы больше значения 20 000 000
data_box_tv =[data[data['year'] == 2015]['Total Volume'], data[data['year'] == 2016]['Total Volume'],data[data['year'] == 2017]['Total Volume']]
plt.boxplot(data_box_tv, labels=['2015','2016','2017'])
plt.title('Общие продажи по годам')
plt.show()
# Посмотрим какому региону они соответствуют
print(data[data['Total Volume']>20000000]['region'].unique())
# Посмотрим для этого региона TotalUS (обощенны данные по США) гистограмму
data[data['region'] == 'TotalUS']['Total Volume'].plot(kind='hist')
plt.title('Рапределение общих продаж по США')
plt.show()
# Посмотрим какой тип авокадо соответствует продажам меньше 10 000 000
print(data[(data['region'] == 'TotalUS') & (data['Total Volume']<10000000)]['type'].unique())
# Получается, что меньшие продажи (< 10 000 000) по типу авокадо organix
# Сравним среднее значение цены по типу авокадо (organix и conventional)
# По графику видно, что средняя цена типа organix выше, видимо, с этим и связаны его более низкие продажи
data[data['type'] == 'organic']['AveragePrice'].plot(kind='hist', alpha=0.7)
data[data['type'] != 'organic']['AveragePrice'].plot(kind='hist', alpha=0.7)
plt.title('Средняя цена по типу авокадо')
plt.xlabel('Средняя цена авокадо')
plt.ylabel('Частота')
plt.legend(['organic', 'conventional'])
plt.show()

# Разделим наш датасет на два, data_totalUS обобщенные данные и data_US данные по регионам США
data_totalUS = data[data['region'] == 'TotalUS']
data_US = data[data['region'] != 'TotalUS']

# На анализ средней цены TotalUS не влияет (предыдущие графики по показателю средней цены корректны).
# По регионам больший размах и большее количество "выбросов",
# что можно объяснить более богатой выборкой по многим регионам не усредненными для обобщенного значения TotalUS
data_box_ap = [data_totalUS['AveragePrice'], data_US['AveragePrice']]
plt.boxplot(data_box_ap, labels=['TotalUS', 'US'])
plt.title('Средняя цена в общем по США(TotalUS) и по регионам')
plt.ylabel('Средняя цена за штуку')
plt.show()
# Но на анализ показателей вроде общего количества проданных штук его влияние заметно
data_box_tv = [data_totalUS['Total Volume'], data_US['Total Volume']]
plt.boxplot(data_box_tv, labels=['TotalUS', 'US'])
plt.ylabel('Общее количество проданных штук')
plt.title('Продажи в общем по США(TotalUS) и по регионам')
plt.show()
# Посмотрим на взаимосвязь показателей Средней цены и Продаж в рарезе типа авокадо
# На основе этих графиков можно сказать, что общие продажи снижаются с увеличением средней цены, срндняя цена отличается
# в зависмости от типа авокадо (organix, conventional), авокадо orgaic покупают в небольших количествах
# рассмотрим подробнее взаимосвязь этих двух показателей далее
sns.pairplot(data_US[['AveragePrice','Total Volume','type']], hue='type')
plt.show()
# Помним, что средняя цена в 2017 году выросла, посмтрим как это отразилось на продажах (на примере датасета регионов)
data_month_US = data_US.groupby(['year','Month']).sum()[['Total Volume']].reset_index()
for x in data['year'].unique():
    plt.plot(data_month_US['Month'].unique(), data_month_US[data_month_US['year'] == x]['Total Volume'])
    plt.title('Динамика продаж авокадо по месяцам и годам')
    plt.xlabel('Месяц')
    plt.ylabel('Количество штук')
    plt.legend(data_month_US['year'].unique())
plt.show()
# Продажи снизились, в том же периоде 2017 года, когда выросла цена
# Построим точечную диаграмму, чтобы посмотреть взаимосязь средней цены и продаж в 2017. На графике видно, что чем ниже
#средняя цена, тем большее количество продаж и, наоборот
plt.scatter(x=data_US[data_US['year']==2017]['AveragePrice'],
            y=data_US[data_US['year']==2017]['Total Volume'],)
plt.xlabel('Цена за штуку')
plt.ylabel('Количество проданных штук')
plt.title('Зависимость продаж от цены')
plt.show()
# На общем графике за 2015, 2016, 2017 подтверждается утверждение об обратной пропорциональности значений цены и продаж
for x in data['year'].unique():
    plt.scatter(x=data_US[data_US['year']==x]['AveragePrice'],
            y=data_US[data_US['year']==x]['Total Volume'], alpha=0.7)
    plt.xlabel('Цена за штуку')
    plt.ylabel('Количество проданных штук')
    plt.legend(data_US['year'].unique())
    plt.title('Зависимость продаж от цены по годам')
plt.show()
# Значение общих продаж Total Volume складывается из продаж авокадо разного размера (маленькие 4046, большие 4225, и супер-большие 4770)
# и упаковок авокадо ('Total Bags')
# Посмотроим как меняются продажи в разрезе этих значений по годам
# Можно сделать вывод, что продажи в общем растут, скорее засчет роста продаж в упаковках
data_US.groupby('year').sum()[['4046', '4225', '4770', 'Total Bags']].plot(kind='bar', stacked=True)
plt.title('Детализация продаж поштучно и в упаковке')
plt.show()
# При этом видно, что выросли в основном продажи маленьких и средних упаковок
data_US.groupby('year').sum()[['Small Bags', 'Large Bags', 'XLarge Bags']].plot(kind='bar', stacked=True)
plt.title('Детализация продаж упакованных авокадо')
plt.show()

